# Milestone 3 - List of S&P 500 companies website

In [2]:
# Load libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# Get html page
html_document = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#:~:text=S%26P%20500%20component%20stocks%20%20%20%20Symbol,%20%20%20%2014%20more%20rows%20')

In [4]:
# Pares through Beautiful Soup
soup = BeautifulSoup(html_document.text, 'html.parser')

In [5]:
# Find all tables
all_tables = soup.find_all("table")
print("Number of tables: {}".format(len(all_tables)))

Number of tables: 2


In [6]:
# the table has id of constituents
table = soup.find_all('table', {'id':'constituents'})

In [7]:
# Get headers of table
headers = [th.getText().strip() for th in table[0].findAll('th')]
headers

['Symbol',
 'Security',
 'SEC filings',
 'GICS Sector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Date added',
 'CIK',
 'Founded']

In [8]:
# Get rows by getting all tr in tbody
rows = table[0].findAll('tr')[1:]
    
# get text from row in each cell
row_data = [[td.text.strip() for td in tr.findAll('td')] for tr in rows]

In [9]:
# Create dataframe with header and row text
sp_table = pd.DataFrame(row_data, columns=headers)
sp_table.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [10]:
# Describe table
sp_table.describe()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
count,503,503,503,503,503,503,503,503,503
unique,503,503,1,11,123,250,352,500,196
top,MMM,3M,reports,Information Technology,Health Care Equipment,"New York City, New York",,0001564708,1985
freq,1,1,503,76,18,42,44,2,12


# Cleaning Steps

## Step 1: Rename Column Headers

In [13]:
new_column_headers = {'Symbol': 'symbol',
                      'Security': 'security',
                      'SEC filings': 'sec',
                      'GICS Sector': 'gics_sec',
                      'GICS Sub-Industry': 'gics_sub',
                      'Headquarters Location': 'headquarters',
                      'Date added': 'sp_date',
                      'CIK': 'cik',
                      'Founded':'founded'}

In [14]:
# Rename columns based on list above and replace
sp_table.rename(columns=new_column_headers, inplace=True)

In [15]:
# Drop SEC Filings
sp_table.drop(['sec'], axis=1, inplace=True)

In [16]:
sp_table.head()

,symbol,security,gics_sec,gics_sub,headquarters,sp_date,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In the first step, I renamed the columns to make it easier to work with and dropped the SEC filings column since it contained a link to a pdf not necessary for this analysis.

## Step 2: Convert Founded Year to int

In [17]:
# Split founded year to only use the first entered year
sp_table['founded'] = sp_table['founded'].str.split('(').str[0]
sp_table['founded'] = sp_table['founded'].str.split('/').str[0]

In [18]:
# Convert from object to int
sp_table['founded'] = sp_table['founded'].astype(int)

In [19]:
sp_table['founded'].dtype

dtype('int32')

In [20]:
sp_table.head()

,symbol,security,gics_sec,gics_sub,headquarters,sp_date,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


The founded year was found to have multiple years, so the first year was used in order to transform into int. Two formats were used to enter the year and both were accounted for in the transformation.

## Step 3: Convert date added to datetime

In [21]:
# Split date added for first date entered
sp_table['sp_date'] = sp_table['sp_date'].str.split('(').str[0]

In [22]:
# Convert to datetime
sp_table['sp_date'] = pd.to_datetime(sp_table['sp_date'])

In [23]:
sp_table['sp_date'].dtype

dtype('<M8[ns]')

In [24]:
sp_table.head()

,symbol,security,gics_sec,gics_sub,headquarters,sp_date,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


The date the company was added to the S&P 500 was convert to datetime after removing additional dates entered.

## Step 4: Remove duplicates by CIK

In [25]:
# Find duplicate rows
sp_table[sp_table.duplicated(subset='cik')]

,symbol,security,gics_sec,gics_sub,headquarters,sp_date,cik,founded
23,GOOG,Alphabet Inc. (Class C),Communication Services,Interactive Media & Services,"Mountain View, California",2006-04-03,0001652044,1998
208,FOX,Fox Corporation (Class B),Communication Services,Movies & Entertainment,"New York City, New York",2015-09-18,0001754301,2019
337,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,0001564708,2013


In [26]:
# Find original rows before duplicate
sp_table[sp_table.duplicated(subset='cik', keep ='last')]

,symbol,security,gics_sec,gics_sub,headquarters,sp_date,cik,founded
22,GOOGL,Alphabet Inc. (Class A),Communication Services,Interactive Media & Services,"Mountain View, California",2014-04-03,0001652044,1998
207,FOXA,Fox Corporation (Class A),Communication Services,Movies & Entertainment,"New York City, New York",2013-07-01,0001754301,2019
336,NWSA,News Corp (Class A),Communication Services,Publishing,"New York City, New York",2013-08-01,0001564708,2013


In [27]:
# Drop duplicates
# Kept Class A records
sp_table.drop_duplicates(subset='cik', inplace=True)

In [28]:
sp_table['cik'] = sp_table['cik'].astype(int)

Multiple entries for the same company were entered by CIK. I dropped duplicates and kept the first entry for all Class A stock tickers.

## Step 5: Split City and State for Headquarters

In [29]:
# Seperate city and state/province for headquarters
# Use first entry
sp_table['hq_city'] = sp_table['headquarters'].str.split(',', expand = True)[0]
sp_table['hq_state_prov'] = sp_table['headquarters'].str.split(',', expand = True)[1]

In [30]:
# Drop original headquarters
sp_table.drop(['headquarters'], axis=1, inplace=True)

In [31]:
sp_table.head()

,symbol,security,gics_sec,gics_sub,sp_date,cik,founded,hq_city,hq_state_prov
0,MMM,3M,Industrials,Industrial Conglomerates,1976-08-09,66740,1902,Saint Paul,Minnesota
1,AOS,A. O. Smith,Industrials,Building Products,2017-07-26,91142,1916,Milwaukee,Wisconsin
2,ABT,Abbott,Health Care,Health Care Equipment,1964-03-31,1800,1888,North Chicago,Illinois
3,ABBV,AbbVie,Health Care,Pharmaceuticals,2012-12-31,1551152,2013,North Chicago,Illinois
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06,1467373,1989,Dublin,Ireland


In step 5, I split the headquarters from the city/location and the state/province into new columns and dropped the original.

## Step 6: Check for Outliers

In [32]:
# Check date ranges of date added
sp_table['sp_date'].describe(datetime_is_numeric=True)

count                              456
mean     2001-06-12 08:47:22.105263104
min                1957-03-04 00:00:00
25%                1989-04-07 18:00:00
50%                2008-08-10 12:00:00
75%                2016-09-06 12:00:00
max                2023-01-04 00:00:00
Name: sp_date, dtype: object

In [33]:
# Check CIK and Founded
sp_table.describe()

,cik,founded
count,5.000000e+02,500.000000
mean,7.888731e+05,1954.786000
std,5.521311e+05,48.488085
min,1.800000e+03,1784.000000
25%,9.740950e+04,1919.000000
50%,8.830380e+05,1969.500000
75%,1.137113e+06,1993.000000
max,1.932393e+06,2022.000000


In step 6, I describe numeric and datetime fields to determine if there appeared to be any signficant outliers.

# Summary

In [34]:
sp_table.head()

,symbol,security,gics_sec,gics_sub,sp_date,cik,founded,hq_city,hq_state_prov
0,MMM,3M,Industrials,Industrial Conglomerates,1976-08-09,66740,1902,Saint Paul,Minnesota
1,AOS,A. O. Smith,Industrials,Building Products,2017-07-26,91142,1916,Milwaukee,Wisconsin
2,ABT,Abbott,Health Care,Health Care Equipment,1964-03-31,1800,1888,North Chicago,Illinois
3,ABBV,AbbVie,Health Care,Pharmaceuticals,2012-12-31,1551152,2013,North Chicago,Illinois
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06,1467373,1989,Dublin,Ireland


The S&P 500 company was sourced by a open page on wikipedia in reference to the list by S&P Dow Jones Indicies. Modfifications can be made at any time and may not contain the most up to date information. For the purposes of this data analysis, I took the first value for multiple columns to be the date or value to be stored. This should be the most up to date while the second value could be additional locations or times that a company was previously founded. Companies listed may not have a headquarters in the United States such as Accenture that is located in Ireland. The main ethical complications come from the source and that the first entered record was taken in as the source for the purposes of this analysis.